In [ ]:
import geopandas as gpd

In [ ]:
import folium

In [ ]:
# Загружаем данные из файлов .geojson
gdf_apartments = gpd.read_file('apartments.geojson')
gdf_parks = gpd.read_file('park.geojson')
gdf_pitch = gpd.read_file('pitch.geojson')
gdf_kindergarten = gpd.read_file('kindergarten.geojson')
gdf_school = gpd.read_file('school.geojson')
gdf_hospital = gpd.read_file('hospital.geojson')
gdf_supermarket = gpd.read_file('supermarket.geojson')

In [ ]:
# Присваиваем вес "1" или "0" в зависимости от года постройки
gdf_apartments['year_weight'] = gdf_apartments['year_built'].apply(lambda x: 1 if x > 1975 else 0)

In [ ]:
# Присваиваем вес "1" или "0" в зависимости от проведения капитального ремонта
gdf_apartments['major_weight'] = gdf_apartments['major_overhaul'].apply(lambda x: 1 if x == 1 else 0)

In [ ]:
# Создаем буфер вокруг парков с радиусом 1.5 км
gdf_parks['geometry'] = gdf_parks.geometry.buffer(1500)
# Проверяем, попадает ли парк в буфер доступности
gdf_apartments['park_in_buffer'] = gdf_apartments.geometry.apply(lambda x: 1 if gdf_parks.geometry.intersects(x).any() else 0)

<ipython-input-35-a1dc1b56c5c8>:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_parks['geometry'] = gdf_parks.geometry.buffer(1500)


In [ ]:
# Проверяем площадь полигонов в pitch.geojson
total_area_pitch = gdf_pitch.geometry.area.sum()
if total_area_pitch >= 0.1 * 100000:  # 10% от 10 га
    gdf_pitch['area_weight'] = 1
else:
    gdf_pitch['area_weight'] = 0

<ipython-input-36-33bf43bacc13>:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  total_area_pitch = gdf_pitch.geometry.area.sum()


In [ ]:
# Проверяем доступность детского сада в радиусе 300 м
gdf_apartments['kindergarten_in_buffer'] = gdf_apartments.geometry.apply(lambda x: 1
if gdf_kindergarten.geometry.intersects(x.buffer(300)).any() else 0)

In [ ]:
# Проверяем доступность школы в радиусе 750 м
gdf_apartments['school_in_buffer'] = gdf_apartments.geometry.apply(lambda x: 1
if gdf_school.geometry.intersects(x.buffer(750)).any() else 0)

In [ ]:
# Проверяем доступность поликлиники в радиусе 1000 м
gdf_apartments['hospital_in_buffer'] = gdf_apartments.geometry.apply(lambda x: 1
if gdf_hospital.geometry.intersects(x.buffer(1000)).any() else 0)

In [ ]:
# Проверяем доступность сервисов в радиусе 500 м
gdf_apartments['supermarket_in_buffer'] = gdf_apartments.geometry.apply(lambda x: 1
if gdf_supermarket.geometry.intersects(x.buffer(500)).any() else 0)

In [ ]:
# Присваиваем вес за выполнение нематериальных ценностей
gdf_apartments['intangible_weight'] = 0

In [ ]:
# Считаем итоговый балл для каждого здания
gdf_apartments['total_score'] = (gdf_apartments['year_weight'] +
                                  gdf_apartments['park_in_buffer'] +
                                  gdf_apartments['kindergarten_in_buffer'] +
                                  gdf_apartments['school_in_buffer'] +
                                  gdf_apartments['hospital_in_buffer'] +
                                  gdf_apartments['supermarket_in_buffer'] +
                                  gdf_apartments['intangible_weight'])

In [ ]:
# Выводим сводную таблицу баллов
summary_table = gdf_apartments[['full_id', 'year_built', 'total_score']]
print(summary_table)

      full_id  year_built  total_score
0   w30701909        1961            5
1   w30701908        1960            5
2   w30701907        1962            5
3   w30701906        1960            5
4   w30738412        1962            5
..        ...         ...          ...
57  w30701895        1960            5
58  w30701913        1960            5
59  w30701912        1962            5
60  w30701911        1962            5
61  w30701910        1960            5

[62 rows x 3 columns]


In [ ]:
map_center = [59.9343, 30.3351]  # Координаты СПб
m = folium.Map(location=map_center, zoom_start=14)

In [ ]:
# Добавляем здания на карту
for _, row in gdf_apartments.iterrows():
    color = 'red' if row['total_score'] < 5 else 'blue' if row['total_score'] <= 7 else 'green'

    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 1,
            'fillOpacity': 0.6,
        },
        tooltip=folium.Tooltip(f"ID: {row['full_id']}, Год постройки: {row['year_built']}, Общий балл: {row['total_score']}")
    ).add_to(m)


In [ ]:
m.save('map.html')